In [95]:
using IntervalArithmetic
#interval representations for pi and sqrt(2)
pi_n=pi..pi 
q2=sqrt(2..2)

#fourier transform of phi(x), phi_h
function phi_h(xi)
    return pi_n*exp(-q2*pi_n*abs(xi))*cos(q2*pi_n*abs(xi)-pi_n/4)
end

#B(\xi,p)
function B(xi,p)
    term1=4*p*xi^2*abs(phi_h(xi))/(p^2-2*xi^2)
    term2=2*xi^2*exp(-pi_n*p)*(1/(10*(p-q2*xi))+15/(100*(p+q2*xi)^2))
    return term1+term2
end

#H(\xi,n)
function H(xi,n)
    val=pi_n*(-1)^n*exp(-pi_n*abs(n)) #phi_h(n/sqrt(2))
    val=val/q2
    res=2*xi^2*(phi_h(xi)-val)
    return res/((n-q2*xi)^2)
end

lim=-(pi_n/q2)*tanh(pi_n/2) #\sum_{n \in \mathbb{Z}}phi_h(n/sqrt(2)).
#want to show R_low>=lim

eps=2^(-6)


function H_tilde(xi,n)
    diff=abs(xi*q2-n)
    if (diff>=eps || n<0)
        return H(xi,n)
    end
    return -(n^2/2)*q2*pi_n^3*((-1)^n)*exp(-pi_n*n)-(xi^2/6)*diff*exp(pi_n*diff)*8*pi_n^4*exp(-pi_n*q2*xi)
end
          
#lower bound for \mathcal{R}(\xi)
function R_low(xi,p)
    inter=phi_h(xi)-pi_n/q2 #phi_h(\xi)-phi_h(0)
    for n=1:p
        inter=inter+H_tilde(xi,n)+H_tilde(xi,-n)
    end
    return inter-B(xi,p)
end



function check(lower,upper,p)
    beg=lower
    print(beg, "\n")
    last=9.0
    temp=beg..last #iteratively checking R_low(xi,p)>=lim on interval tem[beg,last]
    while beg<upper+1/1024
        res=R_low(temp,p) #range of values of R_low([beg,last])
        iters=0
        while !(((res ∩ lim)==∅) & (res>=lim)) #iteratively decrease last until R_low([temp])>=last
            last=(beg+last)/2
            temp=beg..last
            res=R_low(temp,p)
            iters=iters+1
            if iters%10==0
                print("On iteration ",iters, "\n")
            end
            if iters>=50
                last=nextfloat(beg)
                iters=0
            end
        end
        beg=last
        last=beg+1
        temp=beg..last
        print("Checked up to ",beg, "\n")  
    end
    print("Checked on interval ", "[",lower,",",upper,"]","\n")
end


        

check (generic function with 1 method)

In [ ]:
check(0,9,10000)

In [97]:
function tau1_term(n)
    return (142*pi_n*n^2*exp(-pi_n*n))/(9*q2)
end

function tau2_term(n)
    return n^2*(8*n^2+6*(1+2*n)^2)*pi_n*exp(-pi_n*n)/q2
end

function tail_tau1(b)
    return (142*pi_n*exp(-pi_n*b/3))/(9*q2*(1-exp(-pi_n/3)))
end

function tail_tau2(m)
    return (62*pi_n*exp(-pi_n*m/5))/(q2*(1-exp(-pi_n/5)))
end
    

b=100
m=100
val=tail_tau1(b)+tail_tau2(m)

for n=2:b-1
    val=val+tau1_term(n)
end

for n=5:m-1
    val=val+tau2_term(n)
end
    
val


[0.297849, 0.29785]

In [93]:
(6*8^2+1)/((q2*8+1)^2)

[2.53911, 2.53912]

In [99]:
(5/2)*q2*pi_n*exp(-pi_n)-31/100-2*(370*8^4+25*8^2)*exp(-8*q2*pi_n)

[0.169985, 0.169986]